In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px

# Configure the plotting routines
import pandas as pd
import numpy as np

# Local import for the AnimalShelter CRUD module
import sys
sys.path.append("/home/tuckermiddlet_snhu/Desktop")
from animal_shelter import AnimalShelter

# DO NOT EDIT THIS BLOCK
username = "aacuser"
password = "SNHU1738"
shelter = AnimalShelter(username, password)

# Load all records into DataFrame and drop MongoDB '_id'
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('GraziosoSalvareDashboard')

app.layout = html.Div([
    html.Center(html.H1("SNHU CS-340 Dashboard - Tucker Middleton")),
    html.A(html.Img(src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/SNHU_Penmen_Logo.svg/2560px-SNHU_Penmen_Logo.svg.png",
                    style={'width': '150px'}), href="https://www.snhu.edu"),
    
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '15px'}
        )
    ], style={'padding': '10px'}),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '150px', 'maxWidth': '200px'},
    ),

    html.Div([
        dcc.Graph(id='pie-chart')
    ]),

    html.Div(id='map-id')
])

#############################################
# Filtering Logic
#############################################
def apply_filter(filter_type):
    if filter_type == "water":
        return shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif filter_type == "mountain":
        return shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif filter_type == "disaster":
        return shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })
    else:
        return shelter.read({})

#############################################
# CALLBACKS
#############################################
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_table(filter_value):
    records = apply_filter(filter_value)
    new_df = pd.DataFrame.from_records(records)
    new_df.drop(columns=['_id'], inplace=True)
    return new_df.to_dict('records')

@app.callback(
    Output('pie-chart', 'figure'),
    Input('datatable-id', 'data')
)
def update_pie(data):
    if not data:
        return px.pie(names=["No Data"], values=[1], title="Rescue Dog Breed Distribution")
    
    dff = pd.DataFrame(data)
    breed_counts = dff['breed'].value_counts()
    top_breeds = breed_counts.nlargest(5)
    others = breed_counts.iloc[5:].sum()
    
    labels = top_breeds.index.tolist() + (["Other"] if others > 0 else [])
    values = top_breeds.values.tolist() + ([others] if others > 0 else [])
    
    fig = px.pie(
        names=labels,
        values=values,
        title="Rescue Dog Breed Distribution"
    )
    return fig

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10)

    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    if row >= len(dff):
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10)

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[dff.iloc[row]["location_lat"], dff.iloc[row]["location_long"]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]["location_lat"], dff.iloc[row]["location_long"]],
                      children=[
                          dl.Tooltip(dff.iloc[row]["breed"]),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]["name"])
                          ])
                      ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:10710/
